### Ai Earth Engine 《官方文档》- 应用案例
- 按区域检索与下载Sentine-2 L2A影像
> 哨兵2号（Sentinel-2）是欧洲空间局哥白尼计划下的一个地球观测任务，该任务主要对地球表面进行观测以提供相关遥测服务,如森林监测、土地覆盖变化侦测、天然灾害管理。该计划由2颗相同的卫星哨兵2号A（Sentinel-2A）与B（Sentinel-2B）组成的卫星群。

In [1]:
import aie
# 使用token进行鉴权
aie.Authenticate(token='a1e1845361248013ff7fdf5447a00e38')
# 使用 aliyun access_key_id进行鉴权
# aie.Authenticate(access_key_id="*", access_key_secret="*")
# 使用 aliyun RAM STS 进行鉴权(需要 aiearth-core>=1.0.3)
# aie.Authenticate(access_key_id='*', access_key_secret='*', access_key_security_token='*')
aie.Initialize()

计算资源初始化中，请等待...
计算资源初始化完成.


#### 定义矢量区域

In [ ]:
region = aie.FeatureCollection('China_Province') \
            .filter(aie.Filter.eq('province', '浙江省'))             
map = aie.Map(
    center=region.getCenter(),
    height=300,
    zoom=6
)
vis_params = {
    'color': '#00FF00'
}
map.addLayer(
    region,
    vis_params,
    'region',
    bounds=region.getBounds()
)
map

#### Sentinel-2 数据检索

In [8]:
#定义函数，获取指定时间范围内的过滤Sentinel-2影像集合和中值合成的区域裁剪影像
def s2_collection(start_date, end_date):
    s2 = aie.ImageCollection('SENTINEL_MSIL2A') \
            .filterBounds(region) \
            .filterDate(start_date, end_date) \
            .filter('eo:cloud_cover<20')
    mosaic_image = s2.median().clipToCollection(region)
    return s2, mosaic_image
s2, s2_mosaic = s2_collection('2021-04-01', '2022-08-30')

#### 数据可视化

In [9]:
vis_params = {
    'bands': ['B4', 'B3', 'B2'],   
    'min': 0,
    'max': 3500
}
map.addLayer(
    s2_mosaic,
    vis_params,
    'Ture color',
    bounds=region.getBounds()
)
map

Map(bottom=5538.0, center=[29.15771484375, 120.43212890625], controls=(ZoomControl(options=['position', 'zoom_…

#### 影像导出（离线计算）

In [ ]:
# 导出镶嵌影像
task = aie.Export.image.toAsset(s2_mosaic, 's2_mosaic', 200)
task.start()

### Ai Earth Engine 《官方文档》- 应用案例
- Landsat 系列数据去云
> AIE平台中提供的 Landsat 影像均为 USGS Collection 2 中的数据，该集合使用 QA_PIXEL 波段对云、雪进行描述。因此，可通过位运算和逻辑运算，生成云掩膜，利用 updateMask 函数去除可能是云或者云阴影的区域。此算法适用于 Landsat-5 / 7 / 8 / 9 
> 
> 在Landsat C2数据中，QA_PIXEL波段是一个质量评估波段，用于描述每个像素的质量信息，包括云、雪、云阴影等干扰因素的存在情况。它通过二进制的位来表示不同的质量标志，每个位对应一种特定的质量属性。
> 
> 选择相关位：QA_PIXEL波段中，第3位（bit3）表示云（Cloud），第4位（bit4）表示云阴影（Cloud Shadow）。通过位运算（如bitwiseAnd）与这些位对应的掩码进行比较，可以识别出哪些像素被云或云阴影覆盖。

#### 定义去云算法（function）
1. 为什么需要生成位掩码（bitmask？
> 就像子网掩码一样，存储"质量评估（QA）"波段的标识位,并且除了云、云阴影和雪之外，QA_PIXEL波段中还包含其他质量标志，如传感器饱和、填充像元等。通过生成相应的位掩码，可以提取这些信息，用于进一步的质量评估和数据处理。

In [ ]:
def removeLandsatCloud(image):
    #位运算中的左移操作，用于生成特定的位掩码（bitmask）
    cloudShadowBitMask = (1 << 4)  #位掩码（bitmask）用于标识云影,00010000(假设的，不知道总多少位)
    cloudsBitMask = (1 << 3)  #位掩码（bitmask）用于标识云,00100000(假设的，不知道总多少位)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(aie.Image(cloudShadowBitMask)).eq(aie.Image(0)).And(qa.bitwiseAnd(aie.Image(cloudsBitMask)).eq(aie.Image(0)))
    return image.updateMask(mask)

#### 对遥感影像进行缩放因子校正和偏移量校正
2. 这是遥感影像预处理中的一个重要步骤，用于将影像的数字值（DN值）转换为物理量（如反射率或温度）

In [13]:
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(aie.Image(0.0000275)).add(aie.Image(-0.2))
    thermalBands = image.select('ST_B.*').multiply(aie.Image(0.00341802)).add(aie.Image(149.0))
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

#### Landsat 数据检索

In [19]:
region = aie.FeatureCollection('China_Province') \
            .filter(aie.Filter.eq('province', '浙江省')) 
#定义函数，获取指定时间范围内的过滤Sentinel-2影像集合和中值合成的区域裁剪影像
def s2_collection(start_date, end_date):
    s2 = aie.ImageCollection('LANDSAT_LC08_C02_T1_L2') \
            .filterBounds(region) \
            .filterDate(start_date, end_date) \
            .filter(aie.Filter.lte('eo:cloud_cover', 20.0))
    mosaic_image = s2.median().clipToCollection(region)
    return s2, mosaic_image
dataset, dataset_mosaic = s2_collection('2018-01-01', '2018-10-31')

#### 调用算法与可视化
3. aie.ImageCollection.mosaic将时间序列影像集合中的所有影像镶嵌成一景跨越时空的影像（排序覆盖）

In [20]:
images_no_cloud = dataset.map(removeLandsatCloud).map(applyScaleFactors)
image = images_no_cloud.mosaic().clipToCollection(region)
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}
map = aie.Map(
    center=image.getCenter(),
    height=300,
    zoom=5
)
map.addLayer(
    image,
    vis_params,
    'True Color(432)',
    bounds=image.getBounds()
)
map

Map(center=[29.162989375000002, 120.42838813], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoo…

#### 数据导出（离线计算）

In [21]:
# 导出镶嵌影像
task = aie.Export.image.toAsset(image, 'landsat8 去云与校正-官文案例', 500)
task.start()

### Tips：如何利用掩膜（矩阵）图像进行“与门”逻辑电路更新原影像
> 掩膜（Mask）是一种用于控制图像处理区域或处理过程的特殊图像。它通常是一个与原始图像同样大小的二维矩阵，用于选择性地遮盖或显示图像的特定区域。
>- 基本原理：掩膜通常是一个与原始影像具有相同尺寸的二维数组或栅格数据，其每个像素的值用于决定是否对原始影像的对应像素进行处理。一般情况下，掩膜的像素值为0或1，其中0表示不处理（即排除），1表示要处理（即保留）。
>- 主要作用：掩膜可以用于去除影像中的干扰信息、提取感兴趣区域、进行数据融合等操作

In [3]:
image = aie.Image('S2A_MSIL2A_20220103T024121_N0301_R089_T51RTP_20220103T050347')
map = aie.Map(
    center=image.getCenter(),
    height=300,
    zoom=8
)
# 原始真彩色影像
vis_params_raw = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 1000,
    'max': 2600,
}
map.addLayer(
    image,
    vis_params_raw,
    'raw true color image',
    bounds=image.getBounds()
)
map 

Map(center=[30.211869680864705, 120.45063454877959], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [ ]:
# 单波段掩膜，生成bool值矩阵图像
land_mask = image.select(['B11']).gt(aie.Image(200))
vis_params_mask = {
    'min': 0,
    'max': 1,
}
map.addLayer(
    land_mask,
    vis_params_mask,
    'land mask',
    bounds=image.getBounds()
)
map

Map(bottom=11099.0, center=[29.8553466796875, 120.37445068359375], controls=(ZoomControl(options=['position', …

In [ ]:
#对原始影像做单波段掩膜后使用updateMask()更新影像：
#原始影像中被掩膜值为 0 的像素将变为透明，掩膜值为 1 的像素保持原有值不变
land_image = image.updateMask(land_mask)
map.addLayer(
    land_image,
    vis_params_raw,
    'land image',
    bounds=image.getBounds()
)
map

Map(bottom=5645.4658203125, center=[29.630126953125, 119.7235107421875], controls=(ZoomControl(options=['posit…

In [ ]:
# 多波段掩膜，生成bool值矩阵图像
multi_band_mask = image.select(['B4', 'B3', 'B2']).gt(aie.Image(200))
map.addLayer(
    multi_band_mask,
    vis_params_mask,
    'multi-band mask',
    bounds=image.getBounds()
)
map

Map(bottom=11212.0, center=[29.234619140625, 112.28302001953125], controls=(ZoomControl(options=['position', '…

In [ ]:
#对原始影像做多波段掩膜后使用updateMask()更新影像：
#原始影像中被掩膜值为 0 的像素将变为透明，掩膜值为 1 的像素保持原有值不变
multi_band_masked_image = image.select(['B4', 'B3', 'B2']).updateMask(multi_band_mask)
map.addLayer(
    multi_band_masked_image,
    vis_params_raw,
    'multi-band masked image',
    bounds=image.getBounds()
)
map

Map(bottom=11079.2900390625, center=[29.9652099609375, 119.73724365234375], controls=(ZoomControl(options=['po…